# 1. Keras - General Structure for a Convolutional Neural Network

Sources: 
1. https://towardsdatascience.com/simple-introduction-to-convolutional-neural-networks-cdf8d3077bac
2. https://www.pyimagesearch.com/2018/12/31/keras-conv2d-and-convolutional-layers/    
3. https://stats.stackexchange.com/questions/291820/what-is-the-definition-of-a-feature-map-aka-activation-map-in-a-convolutio
4. https://qph.fs.quoracdn.net/main-qimg-704ab7dc6b6ea6e7e919daab06a63537

This notebook can be seen as a general architecture for using Keras with Neural Networks for classifying images.

Firstly, it is important to outline why we shall be looking at a convolutional neural network for image processing rather than say a standard multilayer perceptron model. There are three distinct problems with MLP's vs. CNN's for image processing. 

1- MLP's are not translation invariant. 
If a cat is in the top right hand corner of the image but then in the bottom left hand corner the MLP when then start 'modify' so that it now predicts cats as being in the bottom left hand corner. As such the model is not a robust approach for classifying images. 

2- MLP's use one perceptron for each input.
This leads to too many weights and overfitting can occur.
If we were to look at a 224x224 image x3 (rgb=red,blue,green for each pixel), that would have to be flattened into a column matrix of 150,528. Not doubt, that means a lot of weights need to be trained.

3- CNN's leverage the fact that nearby pixels are more strongly relate than distant ones. As such it 'understands', for lack of a better phrase, that objects are built up of smaller ones, allowing it to generalise.

In [3]:
# Import the Libraries

In [3]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import numpy as np
from tensorflow import random

In [4]:
# The seed is used to memorise the same random number
seed = 1
np.random.seed(seed)
random.set_seed(seed)

There are two ways to construct Keras models: sequential and functional.

The sequential API allows you to create models layer-by-layer for most problems. It is limited in that it does not allow you to create models that share layers or have multiple inputs or outputs.

Alternatively, the functional API allows you to create models that have a lot more flexibility as you can easily define models where layers connect to more than just the previous and next layers. In fact, you can connect layers to (literally) any other layer. As a result, creating complex networks such as siamese networks and residual networks become possible.

For the purposes of this example I shall be using a sequential neural network as seen below.

In [5]:
# Initiate the classifier and adding layers

In [8]:
classifier =Sequential()
# Here we use 32 filters with a kernal size of 3x3
# The input shape is 64x64 because that is the shape of the image and x3 because every pixel is in rbg (red,blue,green)
classifier.add(Conv2D(32,3,3,input_shape=(64,64,3),activation='relu'))

classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))

classifier.add(MaxPooling2D(2,2))

classifier.add(Flatten())

classifier.add(Dense(128,activation='relu'))

In [9]:
# We continue adding dense layers as long as it improves the accuracy of our model
classifier.add(Dense(128,activation='relu'))
classifier.add(Dense(128,activation='relu'))
classifier.add(Dense(128,activation='relu'))

# Filters Explained

At this point it seems imperative to explain filters and how they give CNN's an advantage over MLP's. Here are some filter facts that can aid in understanding their purpose and how they function.

1- Filters allow us to analyse the effects of nearby pixels

2- A standard filter size is 3x3 or 5x5.

3- Let's say we're looking for a nose. The filter shall move from top left to bottom right searching for the components of a nose. This way it doesn't matter if the nose is in the top left or bottom right or anywhere else for that matter. For each point on the image, a value is calculated based on the filter using a convolution operation. Our filter would tell us how strongly a nose appears, in what locations and how many times they appear. This reduces the number of weights that CNN uses vs. an MLP.

4- Each filter produces a feature map (also known as an activation map). Should it result in a high activation that means that a certain feature was found. The feature map is taken through an activation function. This decides once and for all if a certain feature was found at a certain location. The resulting output can then be referred to as a rectified feature map.

5- We can also use pooling layers in order to select the largest values on the feature maps and use these as inputs to subsequent layers. In theory, any type of operation can be done in pooling layers, but in practice, only max pooling is used because we want to find the outliers — these are when our network sees the feature!

In [10]:
# Compile The network

In [11]:
classifier.compile(optimizer='adam', loss='binary_crossentropy',
metrics=['accuracy'])

In [12]:
# Create training and test data generators.

In [13]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory('pneu/chest_xray/train',
                                        target_size = (64, 64),
                                        batch_size = 32,
                                        class_mode = 'binary')

In [ ]:
test_set = test_datagen.flow_from_directory('pneu/chest_xray/val',
                                        target_size = (64, 64),
                                        batch_size = 32,
                                        class_mode = 'binary')

In [ ]:
classifier.fit_generator(training_set,
                        steps_per_epoch = 10000,
                        epochs = 2,
                        validation_data = test_set,
                        validation_steps = 2500,
                        shuffle=False)

In [ ]:
# Classifying a New Image

In [ ]:
from keras.preprocessing import image
new_image = image.load_img('/test_image.jpg', target_size = (64,64))
new_image 

In [ ]:
# Process the image by converting it into a numpy array using the img_to_array function

In [ ]:
new_image = image.img_to_array(new_image)

new_image = np.expand_dims(new_image, axis = 0)

In [ ]:
result = classifier.predict(new_image)

In [ ]:
if result[0][0] == 1:
    prediction = 'Dog'
else:
    prediction = 'cat'
print(prediction)